In [12]:
# read stl file to get coords and interleave
from stl import mesh
from mpl_toolkits import mplot3d
from matplotlib import pyplot

# Create a new plot
figure = pyplot.figure()
axes = mplot3d.Axes3D(figure)

# Load the STL files and add the vectors to the plot
your_mesh = mesh.Mesh.from_file('cube.stl')
axes.add_collection3d(mplot3d.art3d.Poly3DCollection(your_mesh.vectors))

# Auto scale to the mesh size
scale = your_mesh.points.flatten(-1)
axes.auto_scale_xyz(scale, scale, scale)

# Show the plot to the screen
pyplot.show()

/Users/quinntran/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: DeprecationWarning: Non-string object detected for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead


In [7]:
# read gcode to get coords and interleave
# need dimensions of a slice
# 10 x 10 x 10 mm for many_layers_cube.gcode
import json
import os
import numpy as np
from PIL import Image
from PIL import ImageOps
from skimage import color as skcolor
from skimage import io as skio
from skimage import novice as sknov
from skimage import draw as skdr
import matplotlib.pyplot as plt
from scipy.misc import imresize
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import LineCollection

### CONSTANTS

MATERIAL_1 = "material 1"
MATERIAL_2 = "material 2"
MATERIAL_3 = "material 3"

# noop material
MATERIAL_NOOP = "noop"

# color map for visualizing print
COLOR_MAP = {MATERIAL_1: 'r', MATERIAL_2: 'g', MATERIAL_3: 'b'}
MATERIAL_MAP = {MATERIAL_1: 0, MATERIAL_2: 1, MATERIAL_3: 2}
PSI_MAP = {MATERIAL_1: 4, MATERIAL_2: 5, MATERIAL_3: 6}
EXTRUDER_TO_PIN = {0:4, 1:5, 2:6}

###

class GCommand(object):
    """Class representing a single action of the a microvalve"""

    def __init__(self, x, y, z, material, usecs=100):
        """
        Init
        x, x location in gcode coords
        y, y location in gcode coords
        z, z location in gcode coords
        material, material indicator
        usecs, delay after movement
        """
        self.x = x
        self.y = y
        self.z = z
        self.material = material
        self.usecs = usecs
    
    def __str__(self):
        """Returns gcode representation of command"""
        if self.material == MATERIAL_1:
            return "G0 E10; G1 X{} Y{} ;material: {}\nM400 ;wait for position\nG4 P100\nM430 S{} ;send pulse\n"\
            .format(self.x, self.y, self.material, self.usecs)
        elif self.material == MATERIAL_2:
            return "G0 E20; G1 X{} Y{} ;material: {}\nM400 ;wait for position\nG4 P100\nM430 S{} ;send pulse\n"\
            .format(self.x, self.y, self.material, self.usecs)
        elif self.material == MATERIAL_3:
            return "G0 E30; G1 X{} Y{} ;material: {}\nM400 ;wait for position\nG4 P100\nM430 S{} ;send pulse\n"\
            .format(self.x, self.y, self.material, self.usecs)
        elif self.material == MATERIAL_NOOP:
            return ""

        raise ValueError("Unknown material: {}".format(self.material))

class TCommand(object):
    """Class representing a single action of the a microvalve"""

    def __init__(self, material, prev_pin=4, curr_pin=4, psi=0):
        """
        Init
        prev_material, previous material indicator
        material, material indicator
        psi, user input for pressure regulator
        """
        self.prev_pin = prev_pin
        self.curr_pin = EXTRUDER_TO_PIN[material]
        self.material = material
        self.psi = psi
    
    def __str__(self):
        """Returns gcode representation of command"""
        try:
            if self.prev_pin == -1:
                return "T{}; M42 P{} S{};\n"\
                .format(self.material,\
                        self.curr_pin,\
                        self.psi)
            return "M42 P{} S255; T{}; M42 P{} S{};\n"\
            .format(self.prev_pin,\
                    self.material,\
                    self.curr_pin,\
                    self.psi)
        except:
            raise ValueError("Unknown material: {}".format(self.material))
        
def convert_to_gcode(width, height, start_x, start_y, start_z, z_unit, psi=1, num_layers=1, usecs=600, grid_unit=0.5, flip_flop=True):
    """
    Convert a list of binary images to gcommands. Iterates over each pixel
    and forms a GCommand object

    binary_layers, list of ndarrays of dimension height x width (elements are 0 or 1)
    usecs, delay for GCommand
    grid_unit, conversion of pixel to gcode dimensions in x and y
    z_unit, conversion of pixel to gcode dimensions in z
    start_x, start x location in gcode
    start_y, start y location in gcode
    flip_flop, boolean flip scans of left and right to minimize tracking

    return gcommand_layers, list of lists of GCommand objects
    """
    gcommand_layers = []
    prev_material = MATERIAL_NOOP
    for grid_z in range(num_layers):
        gcommands = []

        for grid_y in range(height):
            if grid_y % 2 == 0 and flip_flop:
                x_iterator = reversed(range(width))
            else:
                x_iterator = range(width)
            for grid_x in x_iterator:
                #pixel = binary_layers[grid_z][grid_y, grid_x]
                material = "material 1" # temporary

                if material is not MATERIAL_NOOP:
                    gcommand = GCommand(grid_x * grid_unit + start_x, \
                                        grid_y * grid_unit + start_y, \
                                        grid_z * z_unit + start_z, \
                                        material, \
                                        usecs)
                    gcommands.append(gcommand)

        gcommand_layers.append(gcommands)

    return gcommand_layers


def write_gcode(layered_content, gcommand_layers, gcode_path, layer_names=None, heatbed_temp=37):
    """
    Convert list of gcommands into .gcode file. Also add start and end commands

    gcommand_layers, list of lists of GCommand objects
    gcode_path, path to write output
    layer_names, names for each layer, defaults to one index naming
    heatbed_temp, start temp UNUSED
    """
    assert heatbed_temp <= 200, "{} > max temp 200".format(heatbed_temp)

    if layer_names is not None:
        assert len(layer_names) == len(gcommand_layers), \
                "not enough names for all layers, remove layer_names to default naming"

    with open(gcode_path, 'w') as gcode_file:
        gcode_file.write(layered_content[0])
        
        for layer_index, gcommands in enumerate(gcommand_layers):
            for gcommand in gcommands:
                gcode_file.write(str(gcommand))
            gcode_file.write(layered_content[layer_index+1])

def main():
    #     # user input
    width, height = 34, 34
    start_x, start_y = 90, 90
    psi = 10
    
    with open('slic3r dual.gcode') as f:
        content = f.readlines()
    layered_content = []
    layer, ctr = "", 0
    found_first_z = False
    found_second_z = False
    
    start_z, z_unit = 0, 0
    
    prev_k, prev_v = -1, -1
    for c in content:
        
        if 'G1 Z' in c and 'G1 Z5' not in c:
            
            #'T0\r\n;layer: 117\n'
            layer = ";layer: " + str(ctr) + "\n"
            layered_content.append(layer)
            layer, ctr = "", ctr+1
            
            if found_first_z is False:
                start_z = float(c.split(" ")[1][1:])
                found_first_z = True
            elif found_first_z is True and found_second_z is False:
                temp = float(c.split(" ")[1][1:])
                if temp != start_z:
                    z_unit = temp - start_z
                    found_second_z = True
        
        switched = False
        for k, v in EXTRUDER_TO_PIN.items():
            #if "T"+str(k)+"\r\n" == c:
            if "T"+str(k)+"\r\n" in c:
                tcommand = TCommand(k, prev_v, v, psi)
                layered_content.append(layer)
                layered_content.append(str(tcommand))
                switched = True
                prev_k = k
                prev_v = v
                layer = ""
                
        layer += c
            
    layer = ";layer: " + str(ctr) + "\n"
    for k, v in EXTRUDER_TO_PIN.items():
        layer += "M42 P" + str(v) +" S255;\n" 
    layered_content.append(layer)
        
    print (start_z, z_unit)

    for l in layered_content:
        print l

    # slice dimensions:
    # width, height = 10 # because gcode units are mm
    # each dot is .3 mm apart

    gcommand_layers = convert_to_gcode(width, height, start_x, start_y, start_z, z_unit, psi, len(filtered_layered_content)-1)
    
    write_gcode(filtered_layered_content, gcommand_layers, 'test.gcode')

if __name__ == '__main__':
    main()


(0.35, 0.30000000000000004)
; generated by Slic3r 1.2.9 on 2017-04-28 at 03:39:38

; external perimeters extrusion width = 0.50mm
; perimeters extrusion width = 0.72mm
; infill extrusion width = 0.72mm
; solid infill extrusion width = 0.72mm
; top infill extrusion width = 0.72mm

; external perimeters extrusion width = 0.50mm
; perimeters extrusion width = 0.72mm
; infill extrusion width = 0.72mm
; solid infill extrusion width = 0.72mm
; top infill extrusion width = 0.72mm

M107
M104 S200 T0 ; set temperature
M104 S200 T1 ; set temperature
G28 ; home all axes
G1 Z5 F5000 ; lift nozzle

M109 S200 T0 ; wait for temperature to be reached
M109 S200 T1 ; wait for temperature to be reached
G21 ; set units to millimeters
G90 ; use absolute coordinates
M82 ; use absolute distances for extrusion
G92 E0

T0; M42 P4 S10;

;layer: 0

;layer: 1

G1 Z0.350 F7800.000
G1 E12.42128 F2400.00000
G92 E0

M42 P4 S255; T1; M42 P5 S10;

;layer: 2

G1 Z0.650 F7800.000
G1 E12.42128 F2400.00000
G92 E0

M42 P5 S

NameError: global name 'filtered_layered_content' is not defined

In [ ]:
# M42 P# S255 (is off)
# M42 P# S### (is on)
# T0 - P4
# T1 - P5
# T2 - P6

# now can only call T# when switch extruders

# how are we editing E again? isn't E already given to us
# T# is already mentioned for dual/multiple extruders
